# GEE demo
Processing multiple pre- and post-hurricane images from the Carribean using Nightlight data and export these to GeoTIFF.
[Code from here.](https://github.com/github-moyu/Nightlight_In_The_Carribean/blob/main/multipleimagetes.ipynb)

The Folium library can be used to display ee.Image objects in an interactive map.

1. Load ROI image from S2
2. Process to Level 2A?
3. Export ROI image?

In [65]:
# NOTE: need to install ee, then earthengine-api or 
# else ee will cause conflicts
# !pip install ee
# !pip install earthengine-api
import ee
ee.Initialize()
import pandas as pd
import geemap
import csv
import time
from datetime import date
import numpy as np
import os, sys
import seaborn as sns
import matplotlib.pyplot as plt
import geemap.colormaps as cm

# Load East Houston images

In [68]:
help(ee.element.Element)

Help on class Element in module ee.element:

class Element(ee.computedobject.ComputedObject)
 |  Element(*args, **kwargs)
 |  
 |  Base class for ImageCollection and FeatureCollection.
 |  
 |  Method resolution order:
 |      Element
 |      ee.computedobject.ComputedObject
 |      ee.encodable.Encodable
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, func, args, opt_varName=None)
 |      Constructs a collection by initializing its ComputedObject.
 |  
 |  copyProperties = Element.copyProperties(*args, **kwargs)
 |      Copies metadata properties from one element to another.
 |      
 |      Args:
 |        destination: The object whose properties to override.
 |        source: The object from which to copy the properties.
 |        properties: The properties to copy.  If omitted, all
 |            ordinary (i.e. non-system) properties are copied.
 |        exclude: The list of properties to exclude when copying all
 |            properties. Must not be 

In [62]:
# format: COPERNICUS/S2/<SENSINGTIME_INGESTIONTIME_TILECODE>
s2 = ee.Image('COPERNICUS/S2/20170901T170521_20170901T170523_T15RTN')
Map = geemap.Map()
# xcenter = -95.3690; ycenter = 29.7604

Map.addLayer(s2, {'bands': ['B8', 'B4', 'B3'], 'max': 4000}, 'FCC')
Map.addLayer(s2, {'bands': ['B4', 'B3', 'B2'], 'max': 4000}, 'RGB')
Map.centerObject(s2, 10)
# Map.setCenter(29.7604, -95.3690, 12)

Map(center=[29.486229031435773, -95.25686652585799], controls=(WidgetControl(options=['position', 'transparent…

In [63]:
xmin = -95.2718904093038
xmax = -95.056963723006
ymin = 29.6946331800269
ymax = 29.8049786818043

#  see if needs 5th point to close
roigeom = ee.Geometry.Polygon([
            [xmin, ymin],
            [xmin, ymax],
            [xmax, ymax],
            [xmax, ymin]
        ])

roi = ee.FeatureCollection(roigeom)
roicenter = roigeom.centroid().getInfo()
Map.addLayer(roi, {}, "ROI")
Map.centerObject(roigeom)
Map

Map(bottom=27336.0, center=[30.012179325334976, -93.79321262461224], controls=(WidgetControl(options=['positio…

In [ ]:
out_dir = os.path.join(os.path('~', 'Documents', 'Oxford', 'DPhil', 'flood_mapping'))
file_name = os.path.join(out_dir,'tempL1C.tif')
geemap.ee_export_image(export,filename=file_name,scale=200,crs='EPSG:3857',region=roigeom)

In [66]:
out_dir=os.path.join('~', 'Documents', 'Oxford', 'DPhil', 'flood_mapping')
out_dir

'~/Documents/Oxford/DPhil/flood_mapping'

In [ ]:
# geemap.ee_export_image_collection_to_drive(collection, folder='export', scale=10)

# Stuff from the tutorial

In [ ]:
def maskS2clouds(image):
    # code from https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR#description
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10  # shift left 10 bits
    cirrusBitMask = 1 << 11  # shift left by 11 bits

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000)

In [19]:
# country of interest + map setup
# East Houston, Hurricane Harvey, 2017-09-01
country = "USA"
start_date = "2017-09-01"
end_date = "2016-09-02"
Map = geemap.Map()

# for Level 2A: ee.ImageCollection("COPERNICUS/S2_SR")
dataset = ee.ImageCollection('COPERNICUS/S2').filterDate('2017-01-01','2017-11-01').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).map(maskS2clouds)

visualization = {
  'min': 0.0,
  'max': 0.3,
  'bands': ['B4', 'B3', 'B2'],
}

Map.setCenter(29.7604, -95.3690, 12)
Map.addLayer(dataset.mean(), visualization, 'RGB')

# Get the number of images.
count = dataset.size()
print('Count: ', str(count.getInfo())+'\n')

Count:  655027



In [27]:
help(dataset.toList(dataset.size()))

Help on List in module ee.ee_list object:

class List(ee.computedobject.ComputedObject)
 |  List(*args, **kwargs)
 |  
 |  An object to represent lists.
 |  
 |  Method resolution order:
 |      List
 |      ee.computedobject.ComputedObject
 |      ee.encodable.Encodable
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, arg)
 |      Construct a list wrapper.
 |      
 |      This constructor accepts the following args:
 |        1) A bare list.
 |        2) A ComputedObject returning a list.
 |      
 |      Args:
 |        arg: The list to wrap.
 |      
 |      Raises:
 |        ee_exception.EEException: On bad input.
 |  
 |  add = List.add(*args, **kwargs)
 |      Appends the element to the end of list.
 |      
 |      Args:
 |        list:
 |        element:
 |  
 |  cat = List.cat(*args, **kwargs)
 |      Concatenates the contents of other onto list.
 |      
 |      Args:
 |        list:
 |        other:
 |  
 |  contains = List.contains(*args, **kw

In [13]:
xmin = -95.2718904093038
xmax = -95.056963723006
ymin = 29.6946331800269
ymax = 29.8049786818043

#  see if needs 5th point to close
roigeom = ee.Geometry.Polygon([
            [xmin, ymin],
            [xmin, ymax],
            [xmax, ymax],
            [xmax, ymin]
        ])

roi = ee.FeatureCollection(roigeom)
roicenter = roigeom.centroid().getInfo()
Map.addLayer(roi, {}, "ROI")
Map.centerObject(roigeom)

In [14]:
Map 

Map(center=[29.749827663870672, -95.1644270661549], controls=(WidgetControl(options=['position', 'transparent_…

In [15]:
eastonhouston = dataset.filterBounds(roi)

before = ee.Image(eastonhouston.filterDate(start=start_date).mean())
before = before.clipToCollection(roi)
after = ee.Image(eastonhouston.filterDate(start=end_date).mean())
after = after.clipToCollection(roi)

Map.addLayer(before, {}, "before")
Map.addLayer(after, {}, "after")

Map

Map(bottom=217028.0, center=[29.749827663870672, -95.1644270661549], controls=(WidgetControl(options=['positio…

In [46]:
help(Map)

Help on Map in module geemap.geemap object:

class Map(ipyleaflet.leaflet.Map)
 |  Map(*args, **kwargs)
 |  
 |  The Map class inherits from ipyleaflet.Map. The arguments you can pass to the Map can be found at https://ipyleaflet.readthedocs.io/en/latest/api_reference/map.html. By default, the Map will add Google Maps as the basemap. Set add_google_map = False to use OpenStreetMap as the basemap.
 |  
 |  Returns:
 |      object: ipyleaflet map object.
 |  
 |  Method resolution order:
 |      Map
 |      ipyleaflet.leaflet.Map
 |      ipywidgets.widgets.domwidget.DOMWidget
 |      ipywidgets.widgets.widget.Widget
 |      ipywidgets.widgets.widget.LoggingHasTraits
 |      traitlets.traitlets.HasTraits
 |      traitlets.traitlets.HasDescriptors
 |      ipyleaflet.leaflet.InteractMixin
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, **kwargs)
 |      Public constructor
 |  
 |  addLayer = add_ee_layer(self, ee_object, vis_params={}, name=None, shown=True, o

In [45]:
Palette= cm.palettes.brg

Vis_params= {
    'min': -20,
    'max': 20,
    'palette': Palette
}

pd=after.subtract(before)
pd=pd.divide(before)
pd=pd.multiply(100)
export=pd.visualize(**{
    'min': -20,
    'max': 20,
    'palette': ['blue','red','green']})

Map.addLayer(pd, Vis_params, 'Difference')
Map.add_colorbar(Vis_params, label='Before and after Hurricane Harvey',layer_name='color bar')
Map

EEException: Image.multiply: If one image has no bands, the other must also have no bands. Got 0 and 1.

In [ ]:
out_dir=os.path.join(os.path.expanduser('~'),'Downloads')
file_name=os.path.join(out_dir,'Testing.tif')
geemap.ee_export_image(export,filename=file_name,scale=200,crs='EPSG:3857',region=roigeom)

In [63]:
Map

Map(center=[17.7009, 83.277], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

In [29]:
s2 = ee.ImageCollection("COPERNICUS/S2_SR")#.select('B1')
eastonhouston = s2.filterDate(start_date, '2022-01-01').filterBounds(roi)

before = ee.Image(s2.filterDate(start=start_date).mean())
before = before.clipToCollection(roi)
after = ee.Image(s2.filterDate(start=end_date).mean())
after = after.clipToCollection(roi)

mmap.addLayer(before, {}, "before")
mmap.addLayer(after, {}, "after")

In [18]:
Palette= cm.palettes.brg

Vis_params= {
    'min': -20,
    'max': 20,
    'palette': Palette
}
pd=after.subtract(before)
pd=pd.divide(before)
pd=pd.multiply(100)
export=pd.visualize(**{
    'min': -20,
    'max': 20,
    'palette': ['blue','red','green']})
mmap.addLayer(pd, Vis_params, 'Difference')
mmap.add_colorbar(Vis_params, label='percentage difference in nightlight',layer_name='color bar')

EEException: Image.multiply: If one image has no bands, the other must also have no bands. Got 0 and 1.

In [58]:
out_dir=os.path.join(os.path.expanduser('~'),'Downloads')
file_name=os.path.join(out_dir,'Testing.tif')
geemap.ee_export_image(export,filename=file_name,scale=200,crs='EPSG:3857',region=roigeom)

Generating URL ...
An error occurred while downloading.


UnboundLocalError: local variable 'r' referenced before assignment